In [1]:
import os 
import sys 
sys.path.append(os.path.dirname(sys.path[0])) 
from refineryframe.tns import *

### Creating example data (exceptionally messy dataframe)

In [2]:
df = pd.DataFrame({
    'num_id' : [1, 2, 3, 4, 5],
    'NumericColumn': [1, -np.inf, np.inf,np.nan, None],
    'NumericColumn_exepted': [1, -996, np.inf,np.nan, None],
    'NumericColumn2': [None, None, 1,None, None],
    'NumericColumn3': [1, 2, 3, 4, 5],
    'DateColumn': pd.date_range(start='2022-01-01', periods=5),
    'DateColumn2': [pd.NaT,pd.to_datetime('2022-01-01'),pd.NaT,pd.NaT,pd.NaT],
    'DateColumn3': ['2122-05-01',
                    '2022-01-01',
                    '2021-01-01',
                    '1000-01-09',
                    '1850-01-09'],
    'CharColumn': ['Fół', None, np.nan, 'nót eXpęćTęd', '']
})

df

,num_id,NumericColumn,NumericColumn_exepted,NumericColumn2,NumericColumn3,DateColumn,DateColumn2,DateColumn3,CharColumn
0,1,1.0,1.0,NaN,1,2022-01-01,NaT,2122-05-01,Fół
1,2,-inf,-996.0,NaN,2,2022-01-02,2022-01-01,2022-01-01,None
2,3,inf,inf,1.0,3,2022-01-03,NaT,2021-01-01,NaN
3,4,NaN,NaN,NaN,4,2022-01-04,NaT,1000-01-09,nót eXpęćTęd
4,5,NaN,NaN,NaN,5,2022-01-05,NaT,1850-01-09,


#### Defining specification for the dataframe

In [3]:
MISSING_TYPES = {'date_not_delivered': '1850-01-09',
                 'date_other_missing_type': '1850-01-08',
                 'numeric_not_delivered': -999,
                 'character_not_delivered': 'missing'}

In [4]:
unexpected_exceptions = {
    "col_names_types": "NONE",
    "missing_values": ["NumericColumn_exepted"],
    "missing_types": "NONE",
    "inf_values": "NONE",
    "date_format": "NONE",
    "duplicates": "ALL",
    "date_range": "NONE",
    "numeric_range": "NONE"
}

In [5]:
replace_dict = {-996 : -999,
                "1000-01-09": "1850-01-09"}

### Initializing TnS class

In [6]:
tns = Refiner(dataframe = df,
          replace_dict = replace_dict,
          unexpected_exceptions_duv = unexpected_exceptions)

##### function for detecting column types

In [7]:
tns.get_type_dict_from_dataframe()

{'num_id': 'int64',
 'NumericColumn': 'float64',
 'NumericColumn_exepted': 'float64',
 'NumericColumn2': 'float64',
 'NumericColumn3': 'int64',
 'DateColumn': 'datetime64[ns]',
 'DateColumn2': 'datetime64[ns]',
 'DateColumn3': 'object',
 'CharColumn': 'object'}

##### adding expected types

In [8]:
types_dict_str = {'num_id' : 'int64', 
                   'NumericColumn' : 'float64', 
                   'NumericColumn_exepted' : 'float64', 
                   'NumericColumn2' : 'float64', 
                   'NumericColumn3' : 'int64', 
                   'DateColumn' : 'datetime64[ns]', 
                   'DateColumn2' : 'datetime64[ns]', 
                   'DateColumn3' : 'datetime64[ns]', 
                   'CharColumn' : 'object'}

##### moulding types

In [9]:
tns.set_types(type_dict = types_dict_str)

In [10]:
tns.get_type_dict_from_dataframe()

{'num_id': 'int64',
 'NumericColumn': 'float64',
 'NumericColumn_exepted': 'float64',
 'NumericColumn2': 'float64',
 'NumericColumn3': 'int64',
 'DateColumn': 'datetime64[ns]',
 'DateColumn2': 'datetime64[ns]',
 'DateColumn3': 'datetime64[ns]',
 'CharColumn': 'object'}

#### Check independent conditions

In [11]:
tns.check_missing_types()
tns.check_missing_values()
tns.check_inf_values()
tns.check_col_names_types()
tns.check_date_format()
tns.check_duplicates()
tns.check_numeric_range()

#### Using the main function to detect unexpected values

In [12]:
tns.detect_unexpected_values(earliest_date = "1920-01-01",
                         latest_date = "DateColumn3")

DEBUG:Refiner:=== checking column names and types
DEBUG:Refiner:=== checking for presence of missing values
DEBUG:Refiner:=== checking for presence of missing types
DEBUG:Refiner:=== checking propper date format
DEBUG:Refiner:=== checking expected date range
DEBUG:Refiner:=== checking for presense of inf values in numeric colums
DEBUG:Refiner:=== checking expected numeric range


In [13]:
tns.duv_score

0.5

#### Using function to replace unexpected values with missing types

In [14]:
tns.replace_unexpected_values(numeric_lower_bound = "NumericColumn3",
                                numeric_upper_bound = 4,
                                earliest_date = "1920-01-02",
                                latest_date = "DateColumn2",
                                unexpected_exceptions = {"irregular_values": "NONE",
                                                            "date_range": "DateColumn",
                                                            "numeric_range": "NONE",
                                                            "capitalization": "NONE",
                                                            "unicode_character": "NONE"})


DEBUG:Refiner:=== replacing missing values in category cols with missing types
DEBUG:Refiner:=== replacing all upper case characters with lower case
DEBUG:Refiner:=== replacing character unicode to latin
DEBUG:Refiner:=== replacing missing values in date cols with missing types
DEBUG:Refiner:=== replacing missing values in numeric cols with missing types
DEBUG:Refiner:=== replacing values outside of expected date range
DEBUG:Refiner:=== replacing values outside of expected numeric range
DEBUG:Refiner:** Usable values in the dataframe:  44.44%
DEBUG:Refiner:** Uncorrected data quality score:  32.22%
DEBUG:Refiner:** Corrected data quality score:  52.57%


In [15]:
tns.dataframe

,num_id,NumericColumn,NumericColumn_exepted,NumericColumn2,NumericColumn3,DateColumn,DateColumn2,DateColumn3,CharColumn
0,1,1.0,1.0,-999.0,1,2022-01-01,1850-01-09,1850-01-09,fol
1,2,-999.0,-999.0,-999.0,2,2022-01-02,2022-01-01,2022-01-01,missing
2,3,-999.0,-999.0,-999.0,3,2022-01-03,1850-01-09,1850-01-09,missing
3,4,-999.0,-999.0,-999.0,4,2022-01-04,1850-01-09,1850-01-09,not expected
4,5,-999.0,-999.0,-999.0,-999,2022-01-05,1850-01-09,1850-01-09,missing


#### Use complex targeted conditions

In [16]:
unexpected_conditions = {
    '1': {
        'description': 'Replace numeric missing with with zero',
        'group': 'regex_columns',
        'features': r'^Numeric',
        'query': "{col} < 0",
        'warning': False,
        'set': 0
    },
    '2': {
        'description': "Clean text column from '-ing' endings and 'not ' beginings",
        'group': 'regex clean',
        'features': ['CharColumn'],
        'query': [r'ing', r'^not.'],
        'warning': False,
        'set': ''
    },
    '3': {
        'description': "Replace numeric values in certain column with zeros if > 2",
        'group': 'multicol mapping',
        'features': ['NumericColumn3'],
        'query': '{col} > 2',
        'warning': False,
        'set': 0
    },
    '4': {
        'description': "Replace strings with values if some part of the string is detected",
        'group': 'string check',
        'features': ['CharColumn'],
        'query': f"CharColumn.str.contains('cted', regex = True)",
        'warning': False,
        'set': 'miss'
    }
    }

In [17]:
tns.replace_unexpected_values(unexpected_conditions = unexpected_conditions)

DEBUG:Refiner:=== replacing missing values in category cols with missing types
DEBUG:Refiner:=== replacing all upper case characters with lower case
DEBUG:Refiner:=== replacing character unicode to latin
DEBUG:Refiner:=== replacing with additional cons
DEBUG:root:Replace numeric missing with with zero
DEBUG:root:Clean text column from '-ing' endings and 'not ' beginings
DEBUG:root:Replace numeric values in certain column with zeros if > 2
DEBUG:root:Replace strings with values if some part of the string is detected
DEBUG:Refiner:=== replacing missing values in date cols with missing types
DEBUG:Refiner:=== replacing missing values in numeric cols with missing types
DEBUG:Refiner:=== replacing values outside of expected date range
DEBUG:Refiner:=== replacing values outside of expected numeric range
DEBUG:Refiner:** Usable values in the dataframe:  82.22%
DEBUG:Refiner:** Uncorrected data quality score:  88.89%
DEBUG:Refiner:** Corrected data quality score:  97.53%


In [18]:
tns.dataframe

,num_id,NumericColumn,NumericColumn_exepted,NumericColumn2,NumericColumn3,DateColumn,DateColumn2,DateColumn3,CharColumn
0,1,1.0,1.0,0.0,1,2022-01-01,1850-01-09,1850-01-09,fol
1,2,0.0,0.0,0.0,2,2022-01-02,2022-01-01,2022-01-01,miss
2,3,0.0,0.0,0.0,0,2022-01-03,1850-01-09,1850-01-09,miss
3,4,0.0,0.0,0.0,0,2022-01-04,1850-01-09,1850-01-09,miss
4,5,0.0,0.0,0.0,0,2022-01-05,1850-01-09,1850-01-09,miss


In [19]:
tns.detect_unexpected_values(unexpected_exceptions = {
    "col_names_types": "NONE",
    "missing_values": "NONE",
    "missing_types": "ALL",
    "inf_values": "NONE",
    "date_format": "NONE",
    "duplicates": "ALL",
    "date_range": "NONE",
    "numeric_range": "NONE"
})

DEBUG:Refiner:=== checking column names and types
DEBUG:Refiner:=== checking for presence of missing values
DEBUG:Refiner:=== checking propper date format
DEBUG:Refiner:=== checking expected date range
DEBUG:Refiner:=== checking for presense of inf values in numeric colums
DEBUG:Refiner:=== checking expected numeric range


#### Scores

In [20]:
print(f'duv_score: {tns.duv_score :.4}')
print(f'ruv_score0: {tns.ruv_score0 :.4}')
print(f'ruv_score1: {tns.ruv_score1 :.4}')
print(f'ruv_score2: {tns.ruv_score2 :.4}')

duv_score: 1.0
ruv_score0: 0.8222
ruv_score1: 0.8889
ruv_score2: 0.9753
